In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
main_path = r'../..'
data_path = main_path+'/data'
import sys
sys.path.append(main_path)
from BayDS.lib.pipeline import *
from BayDS.lib.pipeline.ieee_fraud_nodes import *
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union
from BayDS.lib.io import *
import os

In [3]:
# main_dir = r'd:\Documents\Private\Kaggle\Baydin'
main_dir = main_path
# main_dir = r'f:\my\Prog\kaggle\Baydin'
data_dir = f'{main_dir}/Data'

## Pipeline 1: Processing raw, add NanCounts and reduce mem usage

In [ ]:
p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/01-JoinReduceMem')

p.add_node(IEEEFraudTransactionLoaderNode, None, 'transactions',
           params={
               'input_directory': data_dir
           })

p.add_node(IEEEFraudIdentityLoaderNode, None, 'identity',
           params={
               'input_directory': data_dir
           })

p.add_node(AddNaNCountNode, 'transactions', 'transactions',
           params={
               'name': 'NanTransactionCount'
           })
p.add_node(AddNaNCountNode, 'identity', 'identity',
           params={
               'name': 'NanIdentityCount'
           })
p.add_node(JoinNode,
           ('transactions', 'identity'),
           'data',
           params={
               'on': 'TransactionID'
           })

p.add_node(EraserNode, params={
    'remove_keys': ['transactions', 'identity']
})
p.add_node(ReduceMemoryUsageNode,
           'data', 'data',
           params={
               'verbose': True
           })
p.run(verbose=True)
print(p.data['data'].columns)
p.save_data('pickle')

## Pipeline 2: Add all to our best result

In [4]:
data_dir =f'{main_dir}/Snapshots/1/01-JoinReduceMem'

p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/full-fe')

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'data.pkl'
           })
p.add_node(TimeTransformNode, 'data')
p.add_node(SomeAggregatesFromAnyaNode, 'data')
# todo: check LatestBrowser generation
p.add_node(EmailTransformNode, 'data')

numerical_cols = ['id_%02d' % i for i in range(1,12)] + ["V%d"%i for i in range(1,340)] + ["D%d"%i for i in range(1,16)] + ["C%d"%i for i in range(1,15)]  + ['dist1','TransactionAmt', 'NanIdentityCount', 'NanTransactionCount', '_Weekdays', '_Hours', '_Days', 'Date', 'dist2']
label_cols = ['M1', 'M2', 'M3','M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'card4', 'card6', 'ProductCD'] + ['id_%02d'%i for i in (12,15,16,28,29,32,34,35,36,37,38)]
label_cols += ['id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27',
            'id_30', 'id_31',  'id_33', 'DeviceType', 'DeviceInfo', 'P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 
            'P_emaildomain_bin', 'P_emaildomain_suffix', 'R_emaildomain_bin', 'R_emaildomain_suffix']

#add_from Anya Features
numerical_cols+=['TransactionDT', 'Weekdays', 'Hours', 'Days','DT_M', 'DT_W', 'DT_D','id_02_to_mean_card1', 'id_02_to_mean_card4', 'id_02_to_std_card1', 'id_02_to_std_card4', 'D15_to_mean_card1', 'D15_to_mean_card4', 'D15_to_std_card1', 'D15_to_std_card4', 'D15_to_mean_addr1', 'D15_to_std_addr1', 'TransactionAmt_to_mean_card1', 'TransactionAmt_to_mean_card4', 'TransactionAmt_to_std_card1', 'TransactionAmt_to_std_card4', 'TransactionAmt_decimal','nulls1','screen_width', 'screen_height','TransactionAmt_Log', 'card1_count_full',   'Transaction_day_of_week', 'Transaction_hour', 'id_01_count_dist', 'id_31_count_dist', 'id_33_count_dist', 'id_36_count_dist', 'card2_count_full', 'card3_count_full', 'card4_count_full', 'card5_count_full', 'card6_count_full', 'addr1_count_full', 'addr2_count_full', 'id_36_count_full', 'M_sum', 'M_na', 'ProductCD_target_mean', 'M4_target_mean',  'card1_TransactionAmt_mean', 'card1_TransactionAmt_std', 'card2_TransactionAmt_mean', 'card2_TransactionAmt_std', 'card3_TransactionAmt_mean', 'card3_TransactionAmt_std', 'card5_TransactionAmt_mean', 'card5_TransactionAmt_std', 'uid_TransactionAmt_mean', 'uid_TransactionAmt_std', 'uid2_TransactionAmt_mean', 'uid2_TransactionAmt_std', 'uid3_TransactionAmt_mean', 'uid3_TransactionAmt_std', 'card1_fq_enc', 'card2_fq_enc', 'card3_fq_enc', 'card5_fq_enc', 'C1_fq_enc', 'C2_fq_enc', 'C3_fq_enc', 'C4_fq_enc', 'C5_fq_enc', 'C6_fq_enc', 'C7_fq_enc', 'C8_fq_enc', 'C9_fq_enc', 'C10_fq_enc', 'C11_fq_enc', 'C12_fq_enc', 'C13_fq_enc', 'C14_fq_enc', 'D1_fq_enc', 'D2_fq_enc', 'D3_fq_enc', 'D4_fq_enc', 'D5_fq_enc', 'D6_fq_enc', 'D7_fq_enc', 'D8_fq_enc', 'addr1_fq_enc', 'addr2_fq_enc', 'dist1_fq_enc', 'dist2_fq_enc', 'P_emaildomain_fq_enc', 'R_emaildomain_fq_enc', 'DeviceInfo_fq_enc', 'id_30_fq_enc', 'version_id_30_fq_enc', 'version_id_31_fq_enc', 'id_33_fq_enc', 'uid_fq_enc', 'uid2_fq_enc', 'uid3_fq_enc', 'DT_M_total', 'DT_W_total', 'DT_D_total', 'card1_DT_M', 'card2_DT_M', 'card3_DT_M', 'card5_DT_M', 'uid_DT_M', 'uid2_DT_M', 'uid3_DT_M', 'card1_DT_W', 'card2_DT_W', 'card3_DT_W', 'card5_DT_W', 'uid_DT_W', 'uid2_DT_W', 'uid3_DT_W', 'card1_DT_D', 'card2_DT_D', 'card3_DT_D', 'card5_DT_D', 'uid_DT_D', 'uid2_DT_D', 'uid3_DT_D']
label_cols+=['isNight','lastest_browser', 'device_name', 'device_version', 'OS_id_30', 'version_id_30', 'browser_id_31', 'version_id_31', 'had_id', 'id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1','uid', 'uid2', 'uid3']
strange_cols = ['Transaction_day_of_week', 'Transaction_hour']

p.data['numerical_columns'] = numerical_cols
p.data['categorical_columns'] = label_cols
p.data['useless_columns'] = strange_cols

p.add_node(AddDeviceOSInfoNode, ('data', 'numerical_columns', 'categorical_columns'))
p.add_node(AddCardIdNode, ('data', 'numerical_columns', 'categorical_columns'))
p.add_node(AddNewCardIdNode, ('data', 'numerical_columns', 'categorical_columns'))
p.add_node(AddTemporalAggregates, input_key=('data', 'numerical_columns', 'categorical_columns'),
           params={
               'features':['TransactionAmt', 'C5', 'C8'],
               'group_by': 'new_card_id'
           })
p.save()
p.run()
p.save_data()
# del p
# gc.collect()

---------------------------
0: LoaderNode [2019-09-23 12:33:26]
params:
 {'input_directory': '../../Snapshots/1/01-JoinReduceMem', 'file': 'data.pkl'}
---------------------------
1: TimeTransformNode [2019-09-23 12:33:32]
params:
 None
---------------------------
2: SomeAggregatesFromAnyaNode [2019-09-23 12:33:38]
params:
 None


../..\BayDS\lib\pipeline\ieee_fraud_nodes.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  temp_df[new_column] = temp_df[col].astype(str) + '_' + (temp_df[period]).astype(str)


---------------------------
3: EmailTransformNode [2019-09-23 12:37:54]
params:
 None
---------------------------
4: AddDeviceOSInfoNode [2019-09-23 12:37:56]
params:
 None


../..\BayDS\lib\pipeline\ieee_fraud_nodes.py:635: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  idx = data[data.Browser == browser][data.BrowserVersion.isin(browser_map.keys())].index
c:\python36\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


---------------------------
5: AddCardIdNode [2019-09-23 12:38:42]
params:
 None
---------------------------
6: AddNewCardIdNode [2019-09-23 12:38:46]
params:
 None
5%
10%
15%
20%
25%
30%
35%
40%
50%
55%
60%
100%
5%
10%
15%
20%
25%
30%
35%
40%
45%
50%
55%
60%
65%
70%
75%
80%
85%
90%
95%
---------------------------
7: AddTemporalAggregates [2019-09-23 12:43:47]
params:
 {'features': ['TransactionAmt', 'C5', 'C8'], 'group_by': 'new_card_id'}
TransactionAmt
.
.
.
.
.
.
.
.
C5
.
.
.
.
.
.
.
.
C8
.
.
.
.
.
.
.
.
Saving numerical_columns
Saving categorical_columns
Saving useless_columns
Saving data


## Test new node

In [7]:
add = p.data['data'][['new_card_id', 'start_date']]
add.to_pickle(f'{p.working_folder}/new_card_id.pkl')

In [ ]:
data_dir =f'{main_dir}/Snapshots/1/02-FeatureEngineering'

p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/03-AddBrowser')

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'data.pkl'
           })
p.add_node(LoaderNode, None, 'numerical_columns',
           params={
               'input_directory': data_dir,
               'file': 'numerical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'categorical_columns',
           params={
               'input_directory': data_dir,
               'file': 'categorical_columns.yaml'
           })

p.add_node(AddDeviceOSInfoNode, ('data', 'numerical_columns', 'categorical_columns'))
p.save()
p.run(verbose=True)
p.save_data()

In [ ]:
# data_dir =f'{main_dir}/Snapshots/1/03-AddBrowser'

# p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/03-AddBrowser')

# p.add_node(LoaderNode, None, 'data',
#            params={
#                'input_directory': data_dir,
#                'file': 'data.pkl'
#            })
# p.add_node(LoaderNode, None, 'numerical_columns',
#            params={
#                'input_directory': data_dir,
#                'file': 'numerical_columns.yaml'
#            })
# p.add_node(LoaderNode, None, 'categorical_columns',
#            params={
#                'input_directory': data_dir,
#                'file': 'categorical_columns.yaml'
#            })

# p.add_node(AddCardIdNode, ('data', 'numerical_columns', 'categorical_columns'))
# # p.save()
# p.run(verbose=True)
# p.save_data()

### Rubbish

In [ ]:
# df = p.data['data']
# set_cols = set(numerical_cols) | set(label_cols)
# string = ""
# for col in df.columns:
#     if col not in set_cols:
#         print(col, df[col].dtype)
#         string += f"'{col}', "
# print(string)

In [ ]:
del p
gc.collect()

In [ ]:
# unclassified_cols = ['TransactionDT', 'Weekdays', 'Hours', 'Days', 'isNight', 'DT_M', 'DT_W', 'DT_D', 'id_02_to_mean_card1', 'id_02_to_mean_card4', 'id_02_to_std_card1', 'id_02_to_std_card4', 'D15_to_mean_card1', 'D15_to_mean_card4', 'D15_to_std_card1', 'D15_to_std_card4', 'D15_to_mean_addr1', 'D15_to_std_addr1', 'TransactionAmt_to_mean_card1', 'TransactionAmt_to_mean_card4', 'TransactionAmt_to_std_card1', 'TransactionAmt_to_std_card4', 'TransactionAmt_decimal', 'nulls1', 'lastest_browser', 'device_name', 'device_version', 'OS_id_30', 'version_id_30', 'browser_id_31', 'version_id_31', 'screen_width', 'screen_height', 'had_id', 'TransactionAmt_Log', 'card1_count_full', 'Transaction_day_of_week', 'Transaction_hour', 'id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1', 'id_01_count_dist', 'id_31_count_dist', 'id_33_count_dist', 'id_36_count_dist', 'card2_count_full', 'card3_count_full', 'card4_count_full', 'card5_count_full', 'card6_count_full', 'addr1_count_full', 'addr2_count_full', 'id_36_count_full', 'M_sum', 'M_na', 'ProductCD_target_mean', 'M4_target_mean', 'uid', 'uid2', 'uid3', 'card1_TransactionAmt_mean', 'card1_TransactionAmt_std', 'card2_TransactionAmt_mean', 'card2_TransactionAmt_std', 'card3_TransactionAmt_mean', 'card3_TransactionAmt_std', 'card5_TransactionAmt_mean', 'card5_TransactionAmt_std', 'uid_TransactionAmt_mean', 'uid_TransactionAmt_std', 'uid2_TransactionAmt_mean', 'uid2_TransactionAmt_std', 'uid3_TransactionAmt_mean', 'uid3_TransactionAmt_std', 'card1_fq_enc', 'card2_fq_enc', 'card3_fq_enc', 'card5_fq_enc', 'C1_fq_enc', 'C2_fq_enc', 'C3_fq_enc', 'C4_fq_enc', 'C5_fq_enc', 'C6_fq_enc', 'C7_fq_enc', 'C8_fq_enc', 'C9_fq_enc', 'C10_fq_enc', 'C11_fq_enc', 'C12_fq_enc', 'C13_fq_enc', 'C14_fq_enc', 'D1_fq_enc', 'D2_fq_enc', 'D3_fq_enc', 'D4_fq_enc', 'D5_fq_enc', 'D6_fq_enc', 'D7_fq_enc', 'D8_fq_enc', 'addr1_fq_enc', 'addr2_fq_enc', 'dist1_fq_enc', 'dist2_fq_enc', 'P_emaildomain_fq_enc', 'R_emaildomain_fq_enc', 'DeviceInfo_fq_enc', 'id_30_fq_enc', 'version_id_30_fq_enc', 'version_id_31_fq_enc', 'id_33_fq_enc', 'uid_fq_enc', 'uid2_fq_enc', 'uid3_fq_enc', 'DT_M_total', 'DT_W_total', 'DT_D_total', 'card1_DT_M', 'card2_DT_M', 'card3_DT_M', 'card5_DT_M', 'uid_DT_M', 'uid2_DT_M', 'uid3_DT_M', 'card1_DT_W', 'card2_DT_W', 'card3_DT_W', 'card5_DT_W', 'uid_DT_W', 'uid2_DT_W', 'uid3_DT_W', 'card1_DT_D', 'card2_DT_D', 'card3_DT_D', 'card5_DT_D', 'uid_DT_D', 'uid2_DT_D', 'uid3_DT_D']


In [ ]:
# unclassified_cols1 = [   'id_02_to_mean_card1', 'id_02_to_mean_card4', 'id_02_to_std_card1', 'id_02_to_std_card4', 'D15_to_mean_card1', 'D15_to_mean_card4', 'D15_to_std_card1', 'D15_to_std_card4', 'D15_to_mean_addr1', 'D15_to_std_addr1', 'TransactionAmt_to_mean_card1', 'TransactionAmt_to_mean_card4', 'TransactionAmt_to_std_card1', 'TransactionAmt_to_std_card4', 'TransactionAmt_decimal', 'nulls1', 'lastest_browser', 'device_name', 'device_version', 'OS_id_30', 'version_id_30', 'browser_id_31', 'version_id_31', 'screen_width', 'screen_height', 'had_id', 'TransactionAmt_Log', 'card1_count_full', 'Transaction_day_of_week', 'Transaction_hour', 'id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1', 'id_01_count_dist', 'id_31_count_dist', 'id_33_count_dist', 'id_36_count_dist', 'card2_count_full', 'card3_count_full', 'card4_count_full', 'card5_count_full', 'card6_count_full', 'addr1_count_full', 'addr2_count_full', 'id_36_count_full', 'M_sum', 'M_na', 'ProductCD_target_mean', 'M4_target_mean', 'uid', 'uid2', 'uid3', 'card1_TransactionAmt_mean', 'card1_TransactionAmt_std', 'card2_TransactionAmt_mean', 'card2_TransactionAmt_std', 'card3_TransactionAmt_mean', 'card3_TransactionAmt_std', 'card5_TransactionAmt_mean', 'card5_TransactionAmt_std', 'uid_TransactionAmt_mean', 'uid_TransactionAmt_std', 'uid2_TransactionAmt_mean', 'uid2_TransactionAmt_std', 'uid3_TransactionAmt_mean', 'uid3_TransactionAmt_std', 'card1_fq_enc', 'card2_fq_enc', 'card3_fq_enc', 'card5_fq_enc', 'C1_fq_enc', 'C2_fq_enc', 'C3_fq_enc', 'C4_fq_enc', 'C5_fq_enc', 'C6_fq_enc', 'C7_fq_enc', 'C8_fq_enc', 'C9_fq_enc', 'C10_fq_enc', 'C11_fq_enc', 'C12_fq_enc', 'C13_fq_enc', 'C14_fq_enc', 'D1_fq_enc', 'D2_fq_enc', 'D3_fq_enc', 'D4_fq_enc', 'D5_fq_enc', 'D6_fq_enc', 'D7_fq_enc', 'D8_fq_enc', 'addr1_fq_enc', 'addr2_fq_enc', 'dist1_fq_enc', 'dist2_fq_enc', 'P_emaildomain_fq_enc', 'R_emaildomain_fq_enc', 'DeviceInfo_fq_enc', 'id_30_fq_enc', 'version_id_30_fq_enc', 'version_id_31_fq_enc', 'id_33_fq_enc', 'uid_fq_enc', 'uid2_fq_enc', 'uid3_fq_enc', 'DT_M_total', 'DT_W_total', 'DT_D_total', 'card1_DT_M', 'card2_DT_M', 'card3_DT_M', 'card5_DT_M', 'uid_DT_M', 'uid2_DT_M', 'uid3_DT_M', 'card1_DT_W', 'card2_DT_W', 'card3_DT_W', 'card5_DT_W', 'uid_DT_W', 'uid2_DT_W', 'uid3_DT_W', 'card1_DT_D', 'card2_DT_D', 'card3_DT_D', 'card5_DT_D', 'uid_DT_D', 'uid2_DT_D', 'uid3_DT_D']


In [ ]:
# numerical_cols+=['TransactionDT', 'Weekdays', 'Hours', 'Days','DT_M', 'DT_W', 'DT_D','id_02_to_mean_card1', 'id_02_to_mean_card4', 'id_02_to_std_card1', 'id_02_to_std_card4', 'D15_to_mean_card1', 'D15_to_mean_card4', 'D15_to_std_card1', 'D15_to_std_card4', 'D15_to_mean_addr1', 'D15_to_std_addr1', 'TransactionAmt_to_mean_card1', 'TransactionAmt_to_mean_card4', 'TransactionAmt_to_std_card1', 'TransactionAmt_to_std_card4', 'TransactionAmt_decimal','nulls1','screen_width', 'screen_height','TransactionAmt_Log', 'card1_count_full',   'Transaction_day_of_week', 'Transaction_hour', 'id_01_count_dist', 'id_31_count_dist', 'id_33_count_dist', 'id_36_count_dist', 'card2_count_full', 'card3_count_full', 'card4_count_full', 'card5_count_full', 'card6_count_full', 'addr1_count_full', 'addr2_count_full', 'id_36_count_full', 'M_sum', 'M_na', 'ProductCD_target_mean', 'M4_target_mean',  'card1_TransactionAmt_mean', 'card1_TransactionAmt_std', 'card2_TransactionAmt_mean', 'card2_TransactionAmt_std', 'card3_TransactionAmt_mean', 'card3_TransactionAmt_std', 'card5_TransactionAmt_mean', 'card5_TransactionAmt_std', 'uid_TransactionAmt_mean', 'uid_TransactionAmt_std', 'uid2_TransactionAmt_mean', 'uid2_TransactionAmt_std', 'uid3_TransactionAmt_mean', 'uid3_TransactionAmt_std', 'card1_fq_enc', 'card2_fq_enc', 'card3_fq_enc', 'card5_fq_enc', 'C1_fq_enc', 'C2_fq_enc', 'C3_fq_enc', 'C4_fq_enc', 'C5_fq_enc', 'C6_fq_enc', 'C7_fq_enc', 'C8_fq_enc', 'C9_fq_enc', 'C10_fq_enc', 'C11_fq_enc', 'C12_fq_enc', 'C13_fq_enc', 'C14_fq_enc', 'D1_fq_enc', 'D2_fq_enc', 'D3_fq_enc', 'D4_fq_enc', 'D5_fq_enc', 'D6_fq_enc', 'D7_fq_enc', 'D8_fq_enc', 'addr1_fq_enc', 'addr2_fq_enc', 'dist1_fq_enc', 'dist2_fq_enc', 'P_emaildomain_fq_enc', 'R_emaildomain_fq_enc', 'DeviceInfo_fq_enc', 'id_30_fq_enc', 'version_id_30_fq_enc', 'version_id_31_fq_enc', 'id_33_fq_enc', 'uid_fq_enc', 'uid2_fq_enc', 'uid3_fq_enc', 'DT_M_total', 'DT_W_total', 'DT_D_total', 'card1_DT_M', 'card2_DT_M', 'card3_DT_M', 'card5_DT_M', 'uid_DT_M', 'uid2_DT_M', 'uid3_DT_M', 'card1_DT_W', 'card2_DT_W', 'card3_DT_W', 'card5_DT_W', 'uid_DT_W', 'uid2_DT_W', 'uid3_DT_W', 'card1_DT_D', 'card2_DT_D', 'card3_DT_D', 'card5_DT_D', 'uid_DT_D', 'uid2_DT_D', 'uid3_DT_D']
# label_cols+=['isNight','lastest_browser', 'device_name', 'device_version', 'OS_id_30', 'version_id_30', 'browser_id_31', 'version_id_31', 'had_id', 'id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1','uid', 'uid2', 'uid3']
# strange_cols = ['Transaction_day_of_week', 'Transaction_hour']

## Pipeline 3: Add temporal aggregates (merged)

In [ ]:
data_dir =f'{main_dir}/Snapshots/1/99'

p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/04-Temporal')

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'data.pkl'
           })
p.add_node(LoaderNode, None, 'numerical_columns',
           params={
               'input_directory': data_dir,
               'file': 'numerical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'categorical_columns',
           params={
               'input_directory': data_dir,
               'file': 'categorical_columns.yaml'
           })

p.add_node(AddTemporalAggregates, input_key=('data', 'numerical_columns', 'categorical_columns'),
           params={
               'features':['TransactionAmt']
           }
          )
p.save()
p.run(verbose=True)
p.save_data()

## Encode categorial features

In [ ]:
data_dir =f'{main_dir}/Snapshots/1/99'

p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/05-LabelEncoded')

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'data.pkl'
           })
p.add_node(LoaderNode, None, 'numerical_columns',
           params={
               'input_directory': data_dir,
               'file': 'numerical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'categorical_columns',
           params={
               'input_directory': data_dir,
               'file': 'categorical_columns.yaml'
           })

p.add_node(LabelEncoderNode, input_key=('data', 'numerical_columns', 'categorical_columns'),
           output_key="label_encoded_data"
          )
p.add_node(EraserNode, params={
    'remove_keys': ['data']
})
p.save()
p.run(verbose=True)
p.save_data()

In [ ]:
p.save_data()